In [1]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [2]:
#!pip install stable-baselines3[extra]

In [20]:
import stable_baselines3


In [21]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [22]:
Discrete(3)

Discrete(3)

In [23]:
Box(0,1,shape=(3,3)).sample()

array([[0.03535254, 0.9659977 , 0.92886347],
       [0.64741415, 0.6676258 , 0.23749122],
       [0.02374303, 0.9773917 , 0.6075538 ]], dtype=float32)

In [24]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 84, 142,  52],
       [ 63,  56, 125],
       [185,  25, 219]])

In [25]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([69.03542], dtype=float32))

In [26]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([48.059105], dtype=float32))])

In [27]:
MultiBinary(4).sample()

array([1, 1, 0, 0], dtype=int8)

In [28]:
MultiDiscrete([5,2,2]).sample()

array([3, 0, 1])

# 3. Building an Environment

In [29]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=0, high=100, shape=(1,))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        #print(self.state.shape)  #CLARA
        #print(self.observation_space)
        return self.state

In [30]:
env=ShowerEnv()

In [31]:

env.observation_space #CLARA
env.observation_space.sample()
env.action_space.sample()

0

In [32]:
env.reset()

array([37.])

In [16]:
#from stable_baselines3.common.env_checker import check_env

In [17]:
#check_env(env, warn=True)

# 4. Test Environment

In [18]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    #print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [33]:
env.close()



# 5. Train Model

In [34]:
log_path = os.path.join('Training', 'Logs')

In [35]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

# Random Agent, before training                             #CLARA
evaluate_policy(model, env, n_eval_episodes=100)            #CLARA

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/clarawicharz/opt/anaconda3/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-57.46, 2.5745679249147813)

In [45]:
model.learn(total_timesteps=100000)

Logging to Training/Logs/PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -26.5    |
| time/              |          |
|    fps             | 3828     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | -26.3      |
| time/                   |            |
|    fps                  | 2417       |
|    iterations           | 2          |
|    time_elapsed         | 1          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01285518 |
|    clip_fraction        | 0.103      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.09      |
|    explained_variance   | -0.000268  |
|    learning_

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -16.3        |
| time/                   |              |
|    fps                  | 1871         |
|    iterations           | 11           |
|    time_elapsed         | 12           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0018929662 |
|    clip_fraction        | 0.0392       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.08        |
|    explained_variance   | 0.000575     |
|    learning_rate        | 0.0003       |
|    loss                 | 34.6         |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.00386     |
|    value_loss           | 67.2         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 5.8         |
| time/                   |             |
|    fps                  | 1832        |
|    iterations           | 21          |
|    time_elapsed         | 23          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.008816634 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.05       |
|    explained_variance   | 0.00117     |
|    learning_rate        | 0.0003      |
|    loss                 | 16.4        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.00831    |
|    value_loss           | 36.2        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | 42.5       |
| time/                   |            |
|    fps                  | 1815       |
|    iterations           | 31         |
|    time_elapsed         | 34         |
|    total_timesteps      | 63488      |
| train/                  |            |
|    approx_kl            | 0.01742633 |
|    clip_fraction        | 0.234      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.969     |
|    explained_variance   | 8.95e-05   |
|    learning_rate        | 0.0003     |
|    loss                 | 39.9       |
|    n_updates            | 400        |
|    policy_gradient_loss | 0.00294    |
|    value_loss           | 66.6       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 44          |
| time/                   |             |
|    fps                  | 1808        |
|    iterations           | 41          |
|    time_elapsed         | 46          |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.030021423 |
|    clip_fraction        | 0.222       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.953      |
|    explained_variance   | 8.13e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 33.2        |
|    n_updates            | 500         |
|    policy_gradient_loss | 0.0104      |
|    value_loss           | 65.6        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

# 6. Save Model

In [46]:
#ORIGINAL
#model.save('PPO')

In [47]:
#ORIGINAL
#evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [48]:
# FROM YT - CLARA
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')



In [49]:
model.save(shower_path)

In [50]:
del model

In [51]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [52]:
#evaluate_policy(model,env,n_eval_episodes=10,render=True)

In [53]:
# Random Agent, after training                             #CLARA
evaluate_policy(model, env, n_eval_episodes=100)            #CLARA

(59.38, 0.9249864863877744)

In [ ]:
#total_timesteps = 20000: (-57.76, 2.486443242867209)